<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Démarrage-de-la-session-Spark" data-toc-modified-id="Démarrage-de-la-session-Spark-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Démarrage de la session Spark</a></span><ul class="toc-item"><li><span><a href="#Affichage-des-infos-de-session" data-toc-modified-id="Affichage-des-infos-de-session-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Affichage des infos de session</a></span></li><li><span><a href="#Import-des-librairies" data-toc-modified-id="Import-des-librairies-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Import des librairies</a></span></li></ul></li><li><span><a href="#Définition-des-PATH" data-toc-modified-id="Définition-des-PATH-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Définition des PATH</a></span></li><li><span><a href="#Traitement-des-données" data-toc-modified-id="Traitement-des-données-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Traitement des données</a></span><ul class="toc-item"><li><span><a href="#Chargement-des-données" data-toc-modified-id="Chargement-des-données-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Chargement des données</a></span></li><li><span><a href="#Préparation-du-modèle" data-toc-modified-id="Préparation-du-modèle-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Préparation du modèle</a></span></li><li><span><a href="#Broadcast-des-poids-du-modèle" data-toc-modified-id="Broadcast-des-poids-du-modèle-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Broadcast des poids du modèle</a></span></li><li><span><a href="#Fonction-de-création-de-modèle" data-toc-modified-id="Fonction-de-création-de-modèle-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Fonction de création de modèle</a></span></li><li><span><a href="#Définition-du-processus-de-chargement-d'images-et-featurisation" data-toc-modified-id="Définition-du-processus-de-chargement-d'images-et-featurisation-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>Définition du processus de chargement d'images et featurisation</a></span></li><li><span><a href="#Extraction-des-features" data-toc-modified-id="Extraction-des-features-3.6"><span class="toc-item-num">3.6&nbsp;&nbsp;</span>Extraction des features</a></span><ul class="toc-item"><li><span><a href="#Sauvegarde-des-résultats" data-toc-modified-id="Sauvegarde-des-résultats-3.6.1"><span class="toc-item-num">3.6.1&nbsp;&nbsp;</span>Sauvegarde des résultats</a></span></li></ul></li></ul></li><li><span><a href="#Chargement-des-données-enregistrées-et-validation-du-résultat" data-toc-modified-id="Chargement-des-données-enregistrées-et-validation-du-résultat-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Chargement des données enregistrées et validation du résultat</a></span></li></ul></div>

# Démarrage de la session Spark

In [1]:
# L'exécution de cette cellule démarre l'application Spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
3,application_1707659149520_0004,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Affichage des infos de session

In [2]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
3,application_1707659149520_0004,pyspark,idle,Link,Link,None,✔


## Import des librairies

In [72]:
import pandas as pd
import numpy as np
import boto3
import io
import os
import tensorflow as tf
from PIL import Image
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras import Model
from pyspark.sql.functions import col, udf, pandas_udf, PandasUDFType, element_at, split
from pyspark.ml.feature import PCA, VectorAssembler
from pyspark.ml.linalg import Vectors, VectorUDT
import pyarrow.parquet as pq
from pyspark.sql.types import ArrayType, DoubleType

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
print(tf.__version__)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2.11.0

# Définition des PATH

In [5]:
PATH = 's3://p9-data'
PATH_Data = PATH+'/training/'
PATH_Result = PATH+'/Results/'
print('PATH:        '+\
      PATH+'\nPATH_Data:   '+\
      PATH_Data+'\nPATH_Result: '+PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

PATH:        s3://p9-data
PATH_Data:   s3://p9-data/training/
PATH_Result: s3://p9-data/Results/

# Traitement des données
## Chargement des données

In [18]:
images = spark.read.format("binaryFile") \
  .option("pathGlobFilter", "*.jpg") \
  .option("recursiveFileLookup", "true") \
  .load(PATH_Data)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
images = images.withColumn("label", element_at(split(images["path"], "/"), -2))
print(images.printSchema())
print(images.select("path", "label").show(5, False))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)
 |-- label: string (nullable = true)

None
+------------------------------------------------+--------------+
|path                                            |label         |
+------------------------------------------------+--------------+
|s3://p9-data/training/Raspberry/272_100.jpg     |Raspberry     |
|s3://p9-data/training/Raspberry/160_100.jpg     |Raspberry     |
|s3://p9-data/training/Pineapple Mini/252_100.jpg|Pineapple Mini|
|s3://p9-data/training/Raspberry/269_100.jpg     |Raspberry     |
|s3://p9-data/training/Pineapple Mini/184_100.jpg|Pineapple Mini|
+------------------------------------------------+--------------+
only showing top 5 rows

None

## Préparation du modèle

In [21]:
model = MobileNetV2(weights='imagenet',
                    include_top=True,
                    input_shape=(224, 224, 3))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Broadcast des poids du modèle

In [24]:
brodcast_weights = sc.broadcast(new_model.get_weights())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Fonction de création de modèle

In [25]:
def model_fn():
    """
    Returns a MobileNetV2 model with top layer removed 
    and broadcasted pretrained weights.
    """
    model = MobileNetV2(weights='imagenet',
                        include_top=True,
                        input_shape=(224, 224, 3))
    for layer in model.layers:
        layer.trainable = False
    new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)
    new_model.set_weights(brodcast_weights.value)
    return new_model

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Définition du processus de chargement d'images et featurisation

In [39]:
def preprocess(content):
    """
    Preprocesses raw image bytes for prediction.
    """
    img = Image.open(io.BytesIO(content)).resize([224, 224])
    arr = img_to_array(img)
    return preprocess_input(arr)

def featurize_series(model, content_series):
    """
    Featurize a pd.Series of raw images using the input model.
    :return: a pd.Series of image features
    """
    input = np.stack(content_series.map(preprocess))
    preds = model.predict(input)
    # For some layers, output features will be multi-dimensional tensors.
    # We flatten the feature tensors to vectors for easier storage in Spark DataFrames.
    output = [p.flatten() for p in preds]
    
    return pd.Series(output)

@pandas_udf('array<float>', PandasUDFType.SCALAR_ITER)
def featurize_udf(content_series_iter):
    '''
    This method is a Scalar Iterator pandas UDF wrapping our featurization function.
    The decorator specifies that this returns a Spark DataFrame column of type ArrayType(FloatType).

    :param content_series_iter: This argument is an iterator over batches of data, where each batch
                              is a pandas Series of image data.
    '''
    # With Scalar Iterator pandas UDFs, we can load the model once and then re-use it
    # for multiple data batches.  This amortizes the overhead of loading big models.
    model = model_fn()
    for content_series in content_series_iter:
        yield featurize_series(model, content_series)

def features_reduction(content_df):
    
    pca = PCA(k=2, inputCol="features", outputCol="pca_features")
    model = pca.fit(content_df)
    transformed_content = model.transform(content_df)
    return transformed_content

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Extraction des features

In [27]:
features_df = images.repartition(24).select(col("path"),
                                            col("label"),
                                            featurize_udf("content").alias("features")
                                           )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Réduction des features
### Conversion en vecteurs

In [29]:
array_to_vector = udf(lambda a: Vectors.dense(a), VectorUDT())

df = features_df.withColumn("features", array_to_vector(col("features")))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Application du PCA

In [40]:
reducted_df = features_reduction(df)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Modification des Types des colonnes features et pca_features

In [77]:
# Define a UDF to convert Sparse Vector to Dense Vector
sparse_to_dense_udf = udf(lambda sv: sv.toArray().tolist(), ArrayType(DoubleType()))

# Apply the UDF to create a new column with dense vectors
final_df = reducted_df.withColumn("features_dense", sparse_to_dense_udf("features")) \
    .withColumn("pca_features_dense", sparse_to_dense_udf("pca_features"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [85]:
# Drop the original "features" and "pca_features" columns
final_df = final_df.drop("features", "pca_features")

# Rename the vector columns to the original names
final_df = final_df.withColumnRenamed("features_dense", "features") \
    .withColumnRenamed("pca_features_dense", "pca_features")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Sauvegarde des résultats

In [87]:
final_df.write.mode("overwrite").parquet(PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Chargement des données enregistrées et validation du résultat

In [88]:
# Read Parquet files using pyarrow
table = pq.read_table(PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [96]:
# Convert to Pandas DataFrame
df = table.to_pandas()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [97]:
df.loc[0, "features"].shape

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(1280,)

In [98]:
df.shape

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(1310, 4)

In [99]:
df.head()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                                            path  ...                                pca_features
0       s3://p9-data/training/Lychee/214_100.jpg  ...     [-5.360315049200152, 4.904884564442732]
1         s3://p9-data/training/Cocos/92_100.jpg  ...      [-2.399770660235729, 10.4392285463378]
2  s3://p9-data/training/Cantaloupe 1/44_100.jpg  ...  [-0.2460996104889103, -3.4420575303427534]
3   s3://p9-data/training/Rambutan/r_131_100.jpg  ...   [-1.3280803242186798, 10.575464201656299]
4    s3://p9-data/training/Pepper Red/97_100.jpg  ...        [4.55504331263094, 6.72945053126737]

[5 rows x 4 columns]